# Querying Prefect Orion Database

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import os
from typing import Union

import pandas as pd
from sqlalchemy import create_engine

## About

Examining the Prefect Orion database for flow and task runs.

## User Inputs

In [3]:
db_filepath = f"sqlite:///{os.path.expanduser('~')}/.prefect/orion.db"

In [16]:
def show_sql_df(
    query: str,
    conn,
    show_output: bool = True,
    max_cols: Union[int, None] = 100,
    max_colwidth: Union[int, None] = None,
) -> Union[None, pd.DataFrame]:
    """Fetch and display results of SQL query."""
    try:
        df_query_output = pd.read_sql(query, con=conn)
        print(f"Got {len(df_query_output):,} rows of results")
        if not df_query_output.empty:
            if show_output:
                with pd.option_context(
                    "display.max_columns",
                    max_cols,
                    "display.max_colwidth",
                    max_colwidth,
                ):
                    display(df_query_output)
            return df_query_output
    except Exception as e:
        print(str(e))
    return None

In [5]:
# Connect to all databases (required to perform CRUD operations and submit queries)
# URI = f"{DB_TYPE}+{DB_DRIVER}://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
URI = db_filepath

In [6]:
engine = create_engine(URI)
conn = engine.connect()

## Run Queries

### Show All Tables in Database

In [ ]:
%%time
query = """
        SELECT name
        FROM sqlite_master
        WHERE type='table'
        """
_ = show_sql_df(query, conn)

### Show Five Most Recent Completed Flow Runs Filtered by Flow Parameters

In [ ]:
%%time
query = """
        SELECT *
        FROM (
            SELECT *
            FROM flow_run
            WHERE state_type = 'COMPLETED'
            AND json_array_length(json_extract(parameters, '$.nrows')) > 5
            ORDER BY start_time DESC
            LIMIT 5
        )
        ORDER BY start_time
        """
_ = show_sql_df(query, conn)

### Show All Completed Flow Runs, Filtered by Flow Parameters, and their Corresponding Task Runs and Task Run States

In [26]:
%%time
query = """
        /* FLOW RUNS */
        WITH single_flow_run AS (
            SELECT flow_id,
                   id AS flow_run_id,
                   deployment_id,
                   name AS flow_run_name,
                   start_time AS flow_run_start_time,
                   end_time AS flow_run_end_time,
                   flow_version,
                   parameters AS flow_parameters,
                   flow_runner_type
            FROM flow_run
            -- select flow_run_id that is one of the IDs from the five
            -- flow runs found above
            WHERE flow_run_id = '2fa0f339-76b5-4dbe-a021-9b22ae73f4a6'
            AND state_type = 'COMPLETED'
        ),
        /* TASK RUNS */
        task_runs_for_single_flow_run AS (
            SELECT id AS task_run_id,
                   name AS task_run_name,
                   tags AS task_run_tags,
                   run_count AS task_run_count,
                   start_time AS task_run_start_time,
                   end_time AS task_run_end_time,
                   task_key,
                   task_version,
                   flow_run_id
            FROM task_run
            WHERE flow_run_id = '2fa0f339-76b5-4dbe-a021-9b22ae73f4a6'
            AND state_type = 'COMPLETED'
            ORDER BY task_run_start_time
        ),
        /* TASK RUN STATES */
        task_run_states AS (
            SELECT id AS task_run_state_id,
                   created AS task_run_state_created,
                   updated AS task_run_state_updated,
                   type AS task_run_state_type,
                   timestamp AS task_run_state_timestamp,
                   name AS task_run_state_name,
                   json_extract(data, '$.encoding') AS task_run_state_data_encoding,
                   json_extract(data, '$.blob') AS task_run_state_data_blob,
                   task_run_id
            FROM task_run_state
            WHERE data != 'null'
        ),
        /* SUMMARY VIEW */
        flow_summary AS (
            SELECT flow_run_id,
                   flow_run_name,
                   flow_id,
                   flow_name,
                   flow_tags,
                   flow_version,
                   flow_parameters,
                   flow_runner_type,
                   deployment_id,
                   flow_run_start_time,
                   task_run_id,
                   task_run_name,
                   task_run_tags,
                   task_run_count,
                   task_run_start_time,
                   task_run_end_time,
                   flow_run_end_time,
                   task_key,
                   task_version,
                   task_run_state_id,
                   task_run_state_created,
                   task_run_state_updated,
                   task_run_state_type,
                   task_run_state_timestamp,
                   task_run_state_name,
                   task_run_state_data_encoding,
                   task_run_state_data_blob
            FROM single_flow_run
            LEFT JOIN task_runs_for_single_flow_run USING (flow_run_id)
            LEFT JOIN (
                SELECT id AS flow_id,
                       name AS flow_name,
                       tags AS flow_tags
                FROM flow
            ) AS f USING (flow_id)
            LEFT JOIN task_run_states USING (task_run_id)
            ORDER BY flow_run_start_time, task_run_start_time
        )
        SELECT *
        FROM flow_summary
        """
_ = show_sql_df(query, conn, max_colwidth=50)

Got 18 rows of results


,flow_run_id,flow_run_name,flow_id,flow_name,flow_tags,flow_version,flow_parameters,flow_runner_type,deployment_id,flow_run_start_time,task_run_id,task_run_name,task_run_tags,task_run_count,task_run_start_time,task_run_end_time,flow_run_end_time,task_key,task_version,task_run_state_id,task_run_state_created,task_run_state_updated,task_run_state_type,task_run_state_timestamp,task_run_state_name,task_run_state_data_encoding,task_run_state_data_blob
0,2fa0f339-76b5-4dbe-a021-9b22ae73f4a6,rational-mushroom,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,My Example Flow,[],tutorial_02,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_s...",None,None,2022-04-28 19:54:49.308070,269f692d-9b08-49e6-a84d-4c089e6c46df,My Example Task 1-2db78d59-0,"[""tutorial"", ""tag-test1""]",1,2022-04-28 19:54:49.363497,2022-04-28 19:54:49.393041,2022-04-28 19:54:50.834649,2db78d59cf0fbd292408680d7b2a9b9c,None,8d199a43-3c4b-462b-8c64-338de6fb959e,2022-04-28 19:54:49.400128,2022-04-28 19:54:49.400139,COMPLETED,2022-04-28 19:54:49.393041,Completed,cloudpickle,gAWVWwUAAAAAAACMEXBhbmRhcy5jb3JlLmZyYW1llIwJRG...
1,2fa0f339-76b5-4dbe-a021-9b22ae73f4a6,rational-mushroom,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,My Example Flow,[],tutorial_02,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_s...",None,None,2022-04-28 19:54:49.308070,a4a88c3b-a00c-4464-9dd5-5e4466a462f3,My Example Task 2-08f63f8f-0,"[""tag-test2"", ""tutorial""]",1,2022-04-28 19:54:49.451234,2022-04-28 19:54:49.475047,2022-04-28 19:54:50.834649,08f63f8f865a82f66e01010bb44cd203,None,006d5525-f782-45f9-8056-851d434c0f65,2022-04-28 19:54:49.482427,2022-04-28 19:54:49.482437,COMPLETED,2022-04-28 19:54:49.475047,Completed,cloudpickle,gAWVWwUAAAAAAACMEXBhbmRhcy5jb3JlLmZyYW1llIwJRG...
2,2fa0f339-76b5-4dbe-a021-9b22ae73f4a6,rational-mushroom,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,My Example Flow,[],tutorial_02,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_s...",None,None,2022-04-28 19:54:49.308070,7d58fbf4-9436-40f4-93d1-364ccdbe0dda,My Example Task 3-e9e54783-0,"[""tag-test3"", ""tutorial""]",1,2022-04-28 19:54:49.530016,2022-04-28 19:54:49.552710,2022-04-28 19:54:50.834649,e9e5478330cb1e9f0e904a3caeb0bcfc,None,213a4cb4-8e87-42b5-bb3d-7b8d5d4d6d3e,2022-04-28 19:54:49.560880,2022-04-28 19:54:49.560889,COMPLETED,2022-04-28 19:54:49.552710,Completed,cloudpickle,gAVOLg==\n
3,2fa0f339-76b5-4dbe-a021-9b22ae73f4a6,rational-mushroom,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,My Example Flow,[],tutorial_02,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_s...",None,None,2022-04-28 19:54:49.308070,8ebd26b4-9f7e-45a4-8176-f284cc1eb26c,My Example Task 1-2db78d59-1,"[""tutorial"", ""tag-test1""]",1,2022-04-28 19:54:49.605601,2022-04-28 19:54:49.628163,2022-04-28 19:54:50.834649,2db78d59cf0fbd292408680d7b2a9b9c,None,2b1560c0-2c86-4ce3-9c7e-88338b8e4b52,2022-04-28 19:54:49.634373,2022-04-28 19:54:49.634382,COMPLETED,2022-04-28 19:54:49.628163,Completed,cloudpickle,gAWVmwYAAAAAAACMEXBhbmRhcy5jb3JlLmZyYW1llIwJRG...
4,2fa0f339-76b5-4dbe-a021-9b22ae73f4a6,rational-mushroom,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,My Example Flow,[],tutorial_02,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_s...",None,None,2022-04-28 19:54:49.308070,be9a36c6-a43a-48ee-ab20-fb962e5d9a35,My Example Task 2-08f63f8f-1,"[""tag-test2"", ""tutorial""]",1,2022-04-28 19:54:49.682448,2022-04-28 19:54:49.706491,2022-04-28 19:54:50.834649,08f63f8f865a82f66e01010bb44cd203,None,43a45554-7166-4171-a4ab-813694c2070d,2022-04-28 19:54:49.714821,2022-04-28 19:54:49.714832,COMPLETED,2022-04-28 19:54:49.706491,Completed,cloudpickle,gAWVmwYAAAAAAACMEXBhbmRhcy5jb3JlLmZyYW1llIwJRG...
5,2fa0f339-76b5-4dbe-a021-9b22ae73f4a6,rational-mushroom,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,My Example Flow,[],tutorial_02,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_s...",None,None,2022-04-28 19:54:49.308070,a13026c0-de52-47bf-a6e7-61f434d01293,My Example Task 3-e9e54783-1,"[""tag-test3"", ""tutorial""]",1,2022-04-28 19:54:49.761572,2022-04-28 19:54:49.782730,2022-04-28 19:54:50.834649,e9e5478330

CPU times: user 23.7 ms, sys: 1.98 ms, total: 25.7 ms
Wall time: 23.2 ms


### Note About Showing Deployment Flow Runs Without Work Queues

When a deployment is created it can be shown, along with its corresponding `flow_id`, in the `deployment` table as seen below

In [27]:
%%time
query = """
        SELECT *
        FROM deployment
        """
_ = show_sql_df(query, conn)

Got 1 rows of results


,id,created,updated,name,schedule,is_schedule_active,tags,parameters,flow_data,flow_runner_type,flow_runner_config,flow_id
0,b1560641-974a-4519-a4a9-4a0453a7dae8,2022-04-28 19:09:35.775481,2022-04-28 19:09:35.771330,demo-deployment,null,1,"[""tutorial"", ""test-deploy""]","{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_str"": ""db-loader""}","{""encoding"": ""blockstorage"", ""blob"": ""{\""data\"": \""\\\""/home/elsdes3/.prefect/storage/c5ffa862-1bcf-4b64-b716-019a5269e93f\\\""\"", \""block_id\"": \""a753fe1f-f18e-42d6-9c6b-10654687968b\""}""}",subprocess,"{""env"": {}, ""stream_output"": true, ""condaenv"": null, ""virtualenv"": null}",dbbbb6ed-a38c-4f91-929b-76e0fb040a00


CPU times: user 10.6 ms, sys: 0 ns, total: 10.6 ms
Wall time: 9.23 ms


**(NEEDS TO BE VERIFIED)** It does not appear that this approach of running a flow registers as a flow run in the `flow_run` table in the Orion database in the same way that a non-deployment flow run would. This can be seen below where the `flow_id` associated with the deployment shown above does not appear in the `flow run` table even though the Prefect CLI was used to execute a local flow run for this deployment [per the Prefect Orion tutorial docs](https://orion-docs.prefect.io/tutorials/deployments/#creating-the-deployment). For such a locally executed flow run, the `deployment_id` column should contain the ID of the deployment shown in the `deployment` table earlier, but this column only contains missing values

In [28]:
%%time
query = """
        SELECT id,
               flow_id,
               parameters,
               deployment_id
        FROM flow_run
        WHERE flow_id = 'dbbbb6ed-a38c-4f91-929b-76e0fb040a00'
        """
_ = show_sql_df(query, conn)

Got 19 rows of results


,id,flow_id,parameters,deployment_id
0,241a4959-7701-41a2-b024-42944fc89dab,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": 12, ""load_str"": ""db-loader""}",None
1,122f94f0-68ca-43c8-94c6-82ec02cd73cc,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [12, 15], ""load_str"": ""db-loader""}",None
2,2d7fe59f-688f-4a12-af37-6651a21b505c,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [12, 15], ""load_str"": ""db-loader""}",None
3,4992bd2b-2f9f-41f2-b20c-3a8c364d8c38,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [12, 15], ""load_str"": ""db-loader""}",None
4,af502ba6-3a95-42f6-818c-ce61ccb608da,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [12, 15], ""load_str"": ""db-loader""}",None
5,713c181c-0da5-4e5f-a1fa-d200afa89d38,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_str"": ""db-loader""}",None
6,69b42ef3-b836-460c-be7a-8be0c70f002a,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_str"": ""db-loader""}",None
7,7140ee17-4353-49cd-ad16-2c72b35d08d1,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_str"": ""db-loader""}",None
8,57485528-d2e2-4c2a-be39-563c34536125,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_str"": ""db-loader""}",None
9,76f1bb0c-373f-4bfa-af87-aae6a8c514ac,dbbbb6ed-a38c-4f91-929b-76e0fb040a00,"{""nrows"": [17, 25, 38, 55, 1105, 170], ""load_str"": ""db-loader""}",None


CPU times: user 6.45 ms, sys: 1.52 ms, total: 7.97 ms
Wall time: 6.32 ms


In [29]:
conn.close()
engine.dispose()

## Links

1. (Not done here) [**Changing to PostgreSQL database**](https://orion-docs.prefect.io/concepts/database/#configuring-a-postgresql-database)
   - full setup instructions using Docker container